# Daft llm_generate Structured Outputs Examples (OpenAI)

From https://openai.com/index/introducing-structured-outputs-in-the-api/

In [ ]:
!pip install daft openai pydantic

In [ ]:
import daft
from daft.functions import llm_generate
from dotenv import load_dotenv

load_dotenv()

model_id = "gpt-5-nano-2025-08-07"

### Text Generation

In [ ]:
df = daft.from_pylist([
    {"text":"What is the best thing about Daft dataframes (the python library)?"},
])

df_result = df.with_column("result", llm_generate(
        df["text"], 
        model=model_id,
        provider="openai",
    )
)
df_result.show()

### Pydantic JSON Schema with OpenAI

In [ ]:
from pydantic import BaseModel
from openai import OpenAI


class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: list[Step]
    final_answer: str


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model=model_id,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=MathResponse,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)


### Pydantic JSON Schema with Daft

In [ ]:
df = daft.from_pylist([
    {"text": "solve 8x + 31 = 2"},
])

df_result = df.with_column("result", llm_generate(
        "You are a helpful math tutor." + df["text"], 
        model=model_id,
        provider="openai",
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "math-response",
                "schema": MathResponse.model_json_schema(),
            },
        },
    )
)

df_result_validated = df_result.with_column("pydantic_model_validated", df_result["result"].apply(
    lambda x: MathResponse.model_validate_json(x),
    return_dtype= daft.DataType.python()
))

df_result_validated.show()

### Pydantic Function Tool (Not yet supported) 

In [ ]:
from enum import Enum
from typing import Union

from pydantic import BaseModel

import openai
from openai import OpenAI


class Table(str, Enum):
    orders = "orders"
    customers = "customers"
    products = "products"


class Column(str, Enum):
    id = "id"
    status = "status"
    expected_delivery_date = "expected_delivery_date"
    delivered_at = "delivered_at"
    shipped_at = "shipped_at"
    ordered_at = "ordered_at"
    canceled_at = "canceled_at"


class Operator(str, Enum):
    eq = "="
    gt = ">"
    lt = "<"
    le = "<="
    ge = ">="
    ne = "!="


class OrderBy(str, Enum):
    asc = "asc"
    desc = "desc"


class DynamicValue(BaseModel):
    column_name: str


class Condition(BaseModel):
    column: str
    operator: Operator
    value: Union[str, int, DynamicValue]


class Query(BaseModel):
    table_name: Table
    columns: list[Column]
    conditions: list[Condition]
    order_by: OrderBy


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant. The current date is August 6, 2024. You help users query for the data they are looking for by calling the query function.",
        },
        {
            "role": "user",
            "content": "look up all my orders in may of last year that were fulfilled but not delivered on time",
        },
    ],
    tools=[
        openai.pydantic_function_tool(Query),
    ],
)

print(completion.choices[0].message.tool_calls[0].function.parsed_arguments)

### Chat Completions Parse API - Not yet supported

In [15]:
from pydantic import BaseModel

from openai import OpenAI


class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: list[Step]
    final_answer: str


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model=model_id,
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=MathResponse,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)

[Step(explanation='We start with the equation 8x + 31 = 2.', output='8x + 31 = 2'), Step(explanation='We need to isolate the term containing x. To do this, subtract 31 from both sides of the equation.', output='8x + 31 - 31 = 2 - 31'), Step(explanation='Simplify both sides by performing the subtraction.', output='8x = -29'), Step(explanation='To solve for x, divide both sides by 8 to isolate x.', output='x = -29 / 8')]
x = -29/8
